In [203]:
import json
import os
import statistics

import numpy as np
import pandas as pd
from tqdm import tqdm

tqdm.pandas()
pd.options.display.max_columns = None

In [204]:
scenarios_dir = '../../data/processed/scenarios/'
scenario_dir = 'scenarios_1_1_pub_trans_comfort_dist-up_0_0_pub_trans_punctuality_dist-up_0_0_household_cars_dist-down_0_0/'

scenario_path = os.path.join(scenarios_dir, scenario_dir)

In [205]:
agents_examp = pd.read_csv(os.path.join(scenario_path, 'agents_results_1.csv'))

agents_examp.head()

,Step,AgentID,agent_id,home_region,age_sex,pub_trans_comfort,pub_trans_punctuality,bicycle_infrastr_comfort,pedestrian_inconvenience,household_persons,household_cars,household_bicycles,travels_num
0,0,0,0,184,61-x_K,3.0,4.0,2.0,0.0,3.0,0.0,2.0,0
1,0,1,1,195,45-60_K,2.0,2.0,2.0,3.0,3.0,0.0,3.0,2
2,0,2,2,68,0-5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,0,3,3,160,16-19_M,3.0,2.0,3.0,1.0,3.0,1.0,3.0,2
4,0,4,4,261,45-65_M,3.0,2.0,2.0,0.0,2.0,1.0,2.0,0


In [206]:
travels_examp = pd.read_csv(os.path.join(scenario_path, 'travels_results_1.csv'), index_col=0)

travels_examp.head()

,agent_id,start_region,start_place_type,dest_region,dest_place_type,travel_start_time,transport_mode,is_driver
0,1,195,dom,195,inne,787,2,NaN
1,1,195,inne,195,dom,797,2,NaN
2,3,160,dom,135,szkola,468,1,NaN
3,3,135,szkola,160,dom,478,1,NaN
4,5,205,dom,139,inne,1099,0,0.0


In [207]:
travels_examp.shape

(944739, 8)

In [ ]:
# {
#     ‘grupa_wiek_plec’: {
# 		‘modal_split’: {
# 			‘komunikacja’: {
#                 średnia: średnia liczba przejazdów komunikacją,
#                 ‘std’: wartość
#             }
# 			‘auta_kierowca’: jw
# 			‘auta_nie_kierowca’: jw
# 			itd.
# 		},
# 		‘rozklad na cele podrozy’: {
# 			‘dom’: {
#                 ‘średnia’: średnia liczba podróży do domu
#                 ‘std’: std:
#             }
# 			itd.
# 		},
# 		‘liczba podróży’: {
#             średnia
#         },
#         pub_trans_comfort: {
#             średnia ocena
#         },
#         pub_trans_punctuality: {
#             średnia ocena
#         },
#         bicycle_infrastr_comfort: {
#             średnia ocena
#         },
#         pedestrian_inconvenience: {
#             średnia
#         },
#         household_persons: {
#             średnia
#         },
#         household_cars: {
#             średnia
#         },
#         household_bicycles: {
#             średnia
#         }
# 	}
# }

## Automa

In [194]:
scenarios_dir = '../../data/processed/scenarios/'

# 0 0 0
scenario_dir = 'scenarios_1_1_pub_trans_comfort_dist-up_0_0_pub_trans_punctuality_dist-up_0_0_household_cars_dist-down_0_0/'
out_file = 'asc_scenario_0_0_0.json'

# 0 0 3
# scenario_dir = 'scenarios_2_1_pub_trans_comfort_dist-up_0_0_pub_trans_punctuality_dist-up_0_0_household_cars_dist-down_0_3/'
# out_file = 'asc_scenario_0_0_3.json'

# 3 3 0
# scenario_dir = 'scenarios_10_1_pub_trans_comfort_dist-up_0_3_pub_trans_punctuality_dist-up_0_3_household_cars_dist-down_0_0/'
# out_file = 'asc_scenario_3_3_0.json'

# 3 3 3
# scenario_dir = 'scenarios_11_1_pub_trans_comfort_dist-up_0_3_pub_trans_punctuality_dist-up_0_3_household_cars_dist-down_0_3/'
# out_file = 'asc_scenario_3_3_3.json'

# 0 0 15
# scenario_dir = 'scenarios_9_1_pub_trans_comfort_dist-up_0_0_pub_trans_punctuality_dist-up_0_0_household_cars_dist-down_0_15/'
# out_file = 'asc_scenario_0_0_15.json'

# 15 15 0
# scenario_dir = 'scenarios_5_1_pub_trans_comfort_dist-up_0_15_pub_trans_punctuality_dist-up_0_15_household_cars_dist-down_0_0/'
# out_file = 'asc_scenario_15_15_0.json'

# 15 15 15
# scenario_dir = 'scenarios_6_1_pub_trans_comfort_dist-up_0_15_pub_trans_punctuality_dist-up_0_15_household_cars_dist-down_0_15/'
# out_file = 'asc_scenario_15_15_15.json'

scenario_path = os.path.join(scenarios_dir, scenario_dir)

In [195]:
simulations_files = [f for f in os.listdir(scenario_path) if f.startswith(('agents_results_', 'travels_results_'))]
num_simulations = int(len(simulations_files) / 2)

num_simulations

100

In [196]:
unique_age_sex = [
    '6-15_K',
    '6-15_M',
    '16-19_K',
    '16-19_M',
    '20-24_K',
    '20-24_M',
    '25-44_K',
    '25-44_M',
    '45-60_K',
    '45-65_M',
    '61-x_K',
    '66-x_M'
]

unique_transport_mode = [
    'car_driver',
    'car_passenger',
    'public',
    'pedestrian',
    'bicycle'
]

unique_dest_place_type = [
    'dom', 
    'inne', 
    'praca', 
    'szkola', 
    'uczelnia'
]

other_params = [
    'travels_num',
    'pub_trans_comfort',
    'pub_trans_punctuality',
    'bicycle_infrastr_comfort',
    'pedestrian_inconvenience',
    'household_persons',
    'household_cars',
    'household_bicycles'
]

statistics = [
    'mean', 
    'std',
    'median',
    'list'
]


json_dict = {}

for age_sex_comb in unique_age_sex:
    json_dict[age_sex_comb] = {}

    json_dict[age_sex_comb]['modal_split'] = {}
    for tm in unique_transport_mode:
        json_dict[age_sex_comb]['modal_split'][tm] = {}
        for stat in statistics:
            json_dict[age_sex_comb]['modal_split'][tm][stat] = []
    
    json_dict[age_sex_comb]['dest_place_type'] = {}
    for dpt in unique_dest_place_type:
        json_dict[age_sex_comb]['dest_place_type'][dpt] = {}
        for stat in statistics:
            json_dict[age_sex_comb]['dest_place_type'][dpt][stat] = []
    
    for param in other_params:
        json_dict[age_sex_comb][param] = {}
        for stat in statistics:
            json_dict[age_sex_comb][param][stat] = []

In [197]:
def transport_mode_count(df_travels, age_sex_comb, tm):
    if tm.startswith('car_'):
        if tm == 'car_driver':
            transport_mode = 0
            is_driver = 1.0
        else:
            transport_mode = 0
            is_driver = 0.0

        count = df_travels[
            (df_travels['age_sex'] == age_sex_comb) &
            (df_travels['transport_mode'] == transport_mode) &
            (df_travels['is_driver'] == is_driver)
        ].shape[0]
    else:
        if tm == 'public':
            transport_mode = 1
        elif tm == 'pedestrian':
            transport_mode = 2
        elif tm == 'bicycle':
            transport_mode = 3
        else:
            transport_mode = -1
        
        count = df_travels[
            (df_travels['age_sex'] == age_sex_comb) &
            (df_travels['transport_mode'] == transport_mode)
        ].shape[0]
    
    return count


def dest_place_type_count(df_travels, age_sex_comb, dpt):
    count = df_travels[
        (df_travels['age_sex'] == age_sex_comb) &
        (df_travels['dest_place_type'] == dpt)
    ].shape[0]
    
    return count


def get_travels_num(df_travels, age_sex_comb):
    count = df_travels[
        df_travels['age_sex'] == age_sex_comb
    ].shape[0]
    
    return count


def get_avg_param(df_agents, age_sex_comb, param):
    df_agents_filtred = df_agents[
        df_travels['age_sex'] == age_sex_comb
    ]
    avg = df_agents_filtred[param].mean()

    return avg

In [198]:
simulations_files = [f for f in os.listdir(scenario_path) if f.startswith(('agents_results_', 'travels_results_'))]
num_simulations = int(len(simulations_files) / 2)

# for simulation in tqdm(range(1, 3)):
for simulation in tqdm(range(1, num_simulations+1)):
    agents_file = 'agents_results_' + str(simulation) + '.csv'
    travels_file = 'travels_results_' + str(simulation) + '.csv'
    df_agents = pd.read_csv(os.path.join(scenario_path, agents_file))
    df_travels = pd.read_csv(os.path.join(scenario_path, travels_file), index_col=0)

    df_travels = pd.merge(df_travels, df_agents[['agent_id','age_sex']], on='agent_id', how='left') # left - means agents who have not attempted to travel do not exist in this df

    for age_sex_comb in unique_age_sex:
        for tm in unique_transport_mode:
            count = transport_mode_count(df_travels, age_sex_comb, tm)
            for stat in statistics:
                json_dict[age_sex_comb]['modal_split'][tm][stat].append(count)
        
        for dpt in unique_dest_place_type:
            count = dest_place_type_count(df_travels, age_sex_comb, dpt)
            for stat in statistics:
                json_dict[age_sex_comb]['dest_place_type'][dpt][stat].append(count)
        
        for param in other_params:
            if param == 'travels_num':
                count = get_travels_num(df_travels, age_sex_comb)
                for stat in statistics:
                    json_dict[age_sex_comb][param][stat].append(count)
            else:
                avg = get_avg_param(df_agents, age_sex_comb, param)
                for stat in statistics:
                    json_dict[age_sex_comb][param][stat].append(avg)


100%|██████████| 100/100 [35:32<00:00, 21.33s/it]


In [199]:
import statistics

for age_sex_comb in tqdm(unique_age_sex):
    for tm in unique_transport_mode:
        json_dict[age_sex_comb]['modal_split'][tm]['mean'] = statistics.mean(json_dict[age_sex_comb]['modal_split'][tm]['mean'])
        json_dict[age_sex_comb]['modal_split'][tm]['std'] = statistics.stdev(json_dict[age_sex_comb]['modal_split'][tm]['std'])
        json_dict[age_sex_comb]['modal_split'][tm]['median'] = statistics.median(json_dict[age_sex_comb]['modal_split'][tm]['median'])
    
    for dpt in unique_dest_place_type:
        json_dict[age_sex_comb]['dest_place_type'][dpt]['mean'] = statistics.mean(json_dict[age_sex_comb]['dest_place_type'][dpt]['mean'])
        json_dict[age_sex_comb]['dest_place_type'][dpt]['std'] = statistics.stdev(json_dict[age_sex_comb]['dest_place_type'][dpt]['std'])
        json_dict[age_sex_comb]['dest_place_type'][dpt]['median'] = statistics.median(json_dict[age_sex_comb]['dest_place_type'][dpt]['median'])
    
    for param in other_params:
        json_dict[age_sex_comb][param]['mean'] = statistics.mean(json_dict[age_sex_comb][param]['mean'])
        json_dict[age_sex_comb][param]['std'] = statistics.stdev(json_dict[age_sex_comb][param]['std'])
        json_dict[age_sex_comb][param]['median'] = statistics.median(json_dict[age_sex_comb][param]['median'])


100%|██████████| 12/12 [00:00<00:00, 78.77it/s]


In [200]:
# json_dict['6-15_K']

In [201]:
out_dir = '../../data/processed/mc/scenarios'

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [202]:
out_path = os.path.join(out_dir, out_file)

with open(out_path, 'w') as f:
    json.dump(json_dict, f)